In [0]:
%sh
vmstat 3

In [0]:
orders_sdf = spark.read.csv('/FileStore/tables/orders.csv', header=True, inferSchema=True)
trains_sdf = spark.read.csv('/FileStore/tables/order_products_train.csv', header=True, inferSchema=True)
products_sdf = spark.read.csv('/FileStore/tables/products.csv', header=True, inferSchema=True)
aisles_sdf = spark.read.csv('/FileStore/tables/aisles.csv', header=True, inferSchema=True)
depts_sdf = spark.read.csv('/FileStore/tables/departments.csv', header=True, inferSchema=True)

300MB 이상의 파일을 Databricks cloud storage에 upload 되지 않으므로 order_products_prior.zip 파일을 압축된 채 그대로 로드하고, 이를 pandas의 read_csv()의 option인 compressed를 이용하여 pandas DataFrame으로 로드한 뒤 다시 spark DataFrame으로 변환

In [0]:
%fs 
cp /FileStore/tables/order_products_prior.zip file:/home/order_products_prior.zip 

In [0]:
import pandas as pd

priors_pdf = pd.read_csv('/home/order_products_prior.zip', compression='zip', header=0, sep=',', quotechar='"')
priors_sdf = spark.createDataFrame(priors_pdf)
del priors_pdf # 메모리 절약을 위해 pandas dataframe삭제 

In [0]:
# 로드한 DataFrame을 임시 table/view로 생성. 
orders_sdf.createOrReplaceTempView("orders")
priors_sdf.createOrReplaceTempView("priors")
trains_sdf.createOrReplaceTempView("trains")
products_sdf.createOrReplaceTempView("products")
aisles_sdf.createOrReplaceTempView("aisles")
depts_sdf.createOrReplaceTempView("depts")

In [0]:
# temporary table/view로 등록되어 있는지 확인. 
spark.catalog.listTables()

In [0]:
%sql
drop table if exists order_all_prods

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/order_all_prods

In [0]:
%sql


-- priors와 orders를 조인
create table order_all_prods
as
with priors_trains
as
(
  select 'prior' as gubun, a.* from priors a
  union all
  select 'train' as gubun, a.* from trains a
)
select a.order_id, a.product_id, a.add_to_cart_order, a.reordered
  ,b.user_id, b.eval_set, b.order_number, b.order_dow, b.order_hour_of_day
  ,b.days_since_prior_order
from  priors_trains a, orders b
where a.order_id = b.order_id

In [0]:
%sql
select * from order_all_prods where user_id = 1 order by order_number

### 사용자별(USER_ID)별 주문 건수 관련 분석
* 사용자가 주문을 많이 하는 성향인지, 1회 주문시 많은 상품을 주문하는 성향인지, 특정 상품만 주로 주문하는지에 대한 속성 분석
* 사용자별 주문/상품 건수, 고유주문/고유상품 건수 분석
* 사용자별 1회 주문 시 상품 건수, 고유 주문상품 대비 주문 상품 건수, 사용자별 주문 상품 건수 대비 고유 상품 건수 비율

In [0]:
%sql
-- 사용자별 1회 주문시 평균 주문 상품 건수, 1회 주문시 평균 고유 주문 상품 건수, 사용자별 총 상품 건수 대비 고유 상품 건수 비율 
select user_id, usr_total_cnt, prd_distinct_cnt, order_distinct_cnt
  , usr_total_cnt/order_distinct_cnt as usr_avg_prd_cnt -- 사용자별 1회 주문시 주문 상품 건수(평균)
  , usr_total_cnt/prd_distinct_cnt as usr_avg_unq_prd_cnt -- 사용자별 고유 주문 상품 대비 주문 상품 건수(평균) 
  , prd_distinct_cnt/usr_total_cnt as usr_unq_prd_ratio --사용자별 주문 총 상품 건수 대비 고유 상품 건수 비율
from (
  select user_id
    , count(*) as usr_total_cnt -- 사용자별 상품 주문 건수
    ,count(distinct product_id) prd_distinct_cnt -- 사용자별 고유 상품 주문 건수 
    ,count(distinct order_id) order_distinct_cnt -- 사용자별 고유 주문 건수
  from order_all_prods a group by user_id -- where order_id in (2539329, 2398795, 473747)
)


In [0]:
%sql
with
-- 사용자별 1회 주문시 평균 주문 상품 건수, 1회 주문시 평균 고유 주문 상품 건수, 사용자별 총 상품 건수 대비 고유 상품 건수 비율 
order_usr_grp as
(
  -- 사용자별 1회 주문시 평균 주문 상품 건수, 1회 주문시 평균 고유 주문 상품 건수, 사용자별 총 상품 건수 대비 고유 상품 건수 비율 
  select usr_total_cnt, prd_distinct_cnt, order_distinct_cnt
    , usr_total_cnt/order_distinct_cnt as usr_avg_prd_cnt -- 사용자별 1회 주문시 주문 상품 건수(평균)
    , usr_total_cnt/prd_distinct_cnt as usr_avg_unq_prd_cnt -- 사용자별 고유 주문 상품 대비 주문 상품 건수(평균) 
    , prd_distinct_cnt/usr_total_cnt as usr_unq_prd_ratio --사용자별 주문 총 상품 건수 대비 고유 상품 건수 비율
  from (
    select user_id
      , count(*) as usr_total_cnt -- 사용자별 상품 주문 건수
      ,count(distinct product_id) prd_distinct_cnt -- 사용자별 고유 상품 주문 건수 
      ,count(distinct order_id) order_distinct_cnt -- 사용자별 고유 주문 건수
    from order_all_prods a group by user_id -- where order_id in (2539329, 2398795, 473747)
  )
)
-- end of with 구문
select avg(usr_avg_prd_cnt), min(usr_avg_prd_cnt), max(usr_avg_prd_cnt)
from order_usr_grp

In [0]:
%sql
with
-- 사용자별 1회 주문시 평균 주문 상품 건수, 1회 주문시 평균 고유 주문 상품 건수, 사용자별 총 상품 건수 대비 고유 상품 건수 비율
-- 개별 select 값들의 histogram 확인. 
order_usr_grp as
(
 -- 사용자별 1회 주문시 평균 주문 상품 건수, 1회 주문시 평균 고유 주문 상품 건수, 사용자별 총 상품 건수 대비 고유 상품 건수 비율 
  select usr_total_cnt, prd_distinct_cnt, order_distinct_cnt
    , usr_total_cnt/order_distinct_cnt as usr_avg_prd_cnt -- 사용자별 1회 주문시 주문 상품 건수(평균)
    , usr_total_cnt/prd_distinct_cnt as usr_avg_unq_prd_cnt -- 사용자별 고유 주문 상품 대비 주문 상품 건수(평균) 
    , prd_distinct_cnt/usr_total_cnt as usr_unq_prd_ratio --사용자별 주문 총 상품 건수 대비 고유 상품 건수 비율
  from (
    select user_id
      , count(*) as usr_total_cnt -- 사용자별 상품 주문 건수
      ,count(distinct product_id) prd_distinct_cnt -- 사용자별 고유 상품 주문 건수 
      ,count(distinct order_id) order_distinct_cnt -- 사용자별 고유 주문 건수
    from order_all_prods a group by user_id -- where order_id in (2539329, 2398795, 473747)
  )
)
-- end of with 구문
select floor(usr_avg_prd_cnt/5.0)*5 as bin, count(*) bin_cnt
from order_usr_grp group by floor(usr_avg_prd_cnt/5.0)*5 order by 1 

#### 사용자별 reordered 관련 속성 분석
* 사용자별 reordered 총합(사용자별 REORDERED된 상품 건수)
* 사용자별 reordered 비율(사용자별 전체 주문 건수 대비 사용자별 REORDERED된 상품 건수)

In [0]:
%sql
select user_id, count(*) usr_total_cnt
, sum(reordered) usr_reord_cnt -- 사용자별 reordered된 상품 건수 -- sum(case when reordered=1 then 1 else 0 end)
, sum(case when reordered = 0 then 1 else 0 end) as usr_no_reord_cnt -- 사용자별 reorder 하지 않은 상품 건수. count(*) - sum(reoredred)와 동일.
, avg(reordered) usr_reord_avg -- 사용자별 reordered 비율
from order_all_prods group by user_id
/*
select a.*, usr_reordered_cnt/usr_total_cnt
from (
select user_id, count(*) usr_total_cnt
, sum(case when reordered=1 then 1 else 0 end) usr_reordered_cnt
from order_all_prods group by user_id
) a
*/

In [0]:
%sql
with
order_usr_grp as
(
select user_id, count(*) usr_total_cnt
, sum(reordered) usr_reordered_cnt
, avg(reordered) usr_reordered_avg
from order_all_prods group by user_id
) 
-- end of view 절
select floor(usr_reordered_avg/0.1)*0.1 as bin, count(*) bin_cnt
from order_usr_grp group by floor(usr_reordered_avg/0.1)*0.1 order by 1  
/* 
select min(usr_reordered_cnt), max(usr_reordered_cnt), avg(usr_reordered_cnt)
from order_usr_grp 

-- user별 재주문 건수 histogram, 재주문 건수는 주문을 많이 한 고객일 경우 많아짐.
select floor(usr_reordered_cnt/30)*30 as bin, count(*) bin_cnt
from order_usr_grp group by floor(usr_reordered_cnt/30)*30 order by 1 

-- user별 재주문 비율 histogram
select floor(usr_reordered_avg/0.1)*0.1 as bin, count(*) bin_cnt
from order_usr_grp group by floor(usr_reordered_avg/0.1)*0.1 order by 1 
*/

#### 사용자별 지난 주문이후 걸린 일자(days_since_prior_order)의 평균/최대/최소값 추출, 사용자별 평균 dow, hourofday 추출

In [0]:
%sql
-- user별로 지난 주문이후 걸린 일자의 평균/최대/최소값을 추출
-- avg(days_since_prior_order) 값이 작을 수록 보다 자주 주문을 수행. 
-- instacart는 30일 자동 주문 기능이 있음. 30일이 넘는 데이터는 데이터로 만들지 않은 것으로 보임. 
select user_id, avg(days_since_prior_order) uer_avg_prior_days 
    , max(days_since_prior_order) usr_max_prior_days
    , min(days_since_prior_order) usr_min_prior_days
from order_all_prods group by user_id order by user_id limit 30

In [0]:
%sql
with
order_usr_grp as
( 
select user_id, avg(days_since_prior_order) usr_avg_prior_days
    , max(days_since_prior_order) usr_max_prior_days
    , min(days_since_prior_order) usr_min_prior_days
from order_all_prods group by user_id
)
-- end of with절
select max(usr_avg_prior_days), avg(usr_avg_prior_days), min(usr_avg_prior_days)
from order_usr_grp
/* 
select * from order_usr_grp where usr_max_prior_days > 30

select max(usr_avg_prior_days), avg(usr_avg_prior_days), min(usr_avg_prior_days)
from order_usr_grp 
*/ 

In [0]:
%sql
-- 사용자의 사용자의 평균 order_dow, 사용자의 평균 order_hour_day
select user_id
    , avg(order_dow) usr_avg_order_dow
    , avg(order_hour_of_day) usr_avg_order_hour_of_day
from order_all_prods group by user_id order by user_id limit 10

### 사용자+상품 레벨 속성 분석
* 사용자+상품별 주문건수, reorder건수, reorder비율, 카트에 담는 순서
* 사용자+상품별 평균/최대/최소 이전 주문까지 걸리는 일자

In [0]:
%sql
select * from order_all_prods where user_id = 1 order by order_number

In [0]:
%sql
-- 사용자+상품 레벨 속성 추출.
SELECT user_id, product_id,
    count(*) as up_prd_cnt,  -- 사용자의 개별 상품별 주문 건수
    sum(reordered) as up_reordered_sum, -- 사용자+상품 레벨의 reorder 건수 
    avg(reordered) as up_reoredered_avg, -- 사용자+상품 레벨의 주문별 reorder비율 
    sum(reordered)/count(*) as up_reoredered_avg1,
    avg(add_to_cart_order) as up_avg_cart --사용자+상품레벨에서 보통 cart에 몇번째로 담는가? 
FROM order_all_prods GROUP BY user_id, product_id;

In [0]:
%sql
with user_prd_grp
as
(
SELECT user_id, product_id,
    count(*) as up_prd_cnt,  -- 사용자의 개별 상품별 주문 건수
    sum(reordered) as up_reordered_sum, -- 사용자+상품 레벨의 reorder 건수 
    avg(reordered) as up_reoredered_avg, -- 사용자+상품 레벨의 주문별 reorder비율 
    avg(add_to_cart_order) as up_avg_cart --사용자+상품레벨에서 보통 cart에 몇번째로 담는가? 
FROM order_all_prods GROUP BY user_id, product_id
)
select floor(up_reordered_sum/10.0)*10 as bin, avg(up_reoredered_avg) from user_prd_grp
group by floor(up_reordered_sum/10.0)*10 order by 1

In [0]:
%sql
-- 사용자+상품 레벨 속성 추출.
SELECT user_id, product_id,
    avg(days_since_prior_order) as up_avg_prior_days,
    max(days_since_prior_order) as up_max_prior_days,
    min(days_since_prior_order) as up_min_prior_days,
    avg(order_dow) as up_avg_ord_dow,
    avg(order_hour_of_day) as up_avg_ord_hour
FROM order_all_prods GROUP BY user_id, product_id;